In [1]:

import pandas as pd
import datetime


master_dataset = pd.read_csv('/data/anuj/JA/2016-09-19_79351_training.csv', delimiter = ',')
mcc_group_definition = pd.read_csv('/data/anuj/JA/mcc_group_definition.csv', delimiter = ',')
transaction_types = pd.read_csv('/data/anuj/JA/transaction_types.csv', delimiter = ',')

print "Master Data\n\n==========\n\n",master_dataset.head(2)
print "\n\nMCC Group Data\n\n========\n\n",mcc_group_definition.head(2)
print "\n\nTransaction Types Data\n======\n\n",transaction_types.head(2)

Master Data


                            user_id transaction_date transaction_type  \
0  a78884f5e76951188c1e719d4956773a       2016-02-01               DT   
1  b0333294fef6ff1299102a70ad46b126       2016-02-01               DT   

   mcc_group  amount_n26_currency dataset_transaction dataset_user  
0        NaN                  350            training     training  
1        NaN                  202            training     training  


MCC Group Data


   mcc_group explanation
0          1         ATM
1          2   Groceries


Transaction Types Data

  type            explanation direction agent
0   AR   Authorization reject         -  Card
1   AE  Authorization expired        In  Card


In [2]:
print master_dataset.shape

(408546, 7)


In [3]:


#a = training.query('user_id == "a78884f5e76951188c1e719d4956773a"')

#a = set(training['user_id'])

'''
I am making a model for prdicitng income and expences of individual.

So First making two dataset one is only having income and other is only for expencse.
'''

dict_transaction = {}

for i in range(len(transaction_types['type'])):
    if(transaction_types['direction'][i]=='In'):
                dict_transaction[transaction_types['type'][i]] =1
    if(transaction_types['direction'][i]=='Out'):
                dict_transaction[transaction_types['type'][i]] =-1

In [4]:
print master_dataset.shape

(408546, 7)


In [5]:
dict_transaction.keys()

['FT',
 'DI',
 'DD',
 'WU',
 'DT',
 'DR',
 'PR',
 'PT',
 'BUS',
 'PF',
 'BUB',
 'AA',
 'AE',
 'WEE',
 'AL',
 'AU',
 'AV',
 'BBU',
 'CT',
 'BRU',
 'TBU',
 'TUB']

In [6]:
######### As transaction_types is not blank I am doing inner join ########
######## Could not find data for all transaction type specially for "-" ##########
transaction_types.columns=["transaction_type", 'explanation', 'direction', 'agent']
master_dataset_new=pd.merge(master_dataset,transaction_types,on='transaction_type',how='inner')
#transaction_types.columns.tolist()

In [7]:
########## removing unwanted columns #############
master_dataset_new=master_dataset_new.drop(['explanation','dataset_transaction','dataset_user'], axis=1)
########### replcaing mcc_group's value NAN to 0 #########
master_dataset_new['mcc_group']=master_dataset_new['mcc_group'].fillna(0).astype(int)
print master_dataset_new.columns.tolist()
master_dataset_new['mcc_group'].value_counts()

['user_id', 'transaction_date', 'transaction_type', 'mcc_group', 'amount_n26_currency', 'direction', 'agent']


0     150222
1      69807
2      50586
6      24672
3      20875
8      20610
4      20194
17     11466
16     11150
7      10888
10      7282
11      5803
5       3337
12       873
15       329
9        287
13       141
14        24
Name: mcc_group, dtype: int64

In [8]:
master_dataset_new['direction'].value_counts()


Out    336421
In      72125
Name: direction, dtype: int64

In [9]:
master_dataset_new.head()

,user_id,transaction_date,transaction_type,mcc_group,amount_n26_currency,direction,agent
0,a78884f5e76951188c1e719d4956773a,2016-02-01,DT,0,350,Out,Bank Account
1,b0333294fef6ff1299102a70ad46b126,2016-02-01,DT,0,202,Out,Bank Account
2,7b8d2a2780adae0cd0c248e92c1b28dc,2016-02-01,DT,0,291,Out,Bank Account
3,cc4abaa500f7db4390ae3f02bd36d805,2016-02-01,DT,0,214,Out,Bank Account
4,49b99d1d5ba028566639e8b3eb7c055b,2016-02-01,DT,0,272,Out,Bank Account


In [10]:
############# converting index for string features such as transaction_type and agent ########
transaction_type_map=pd.DataFrame(master_dataset_new.transaction_type.unique().tolist()).reset_index()
transaction_type_map.columns=['transaction_type_Id','transaction_type']
print transaction_type_map
agent_map=pd.DataFrame(master_dataset_new.agent.unique().tolist()).reset_index()
agent_map.columns=['agent_Id','agent']
print agent_map

   transaction_type_Id transaction_type
0                    0               DT
1                    1               CT
2                    2               DR
3                    3               FT
4                    4               DD
5                    5               PT
6                    6              BBU
7                    7              BUB
8                    8              TUB
   agent_Id         agent
0         0  Bank Account
1         1          Card
2         2       Partner


In [11]:
########### Replacing the values ###############
master_dataset_new1=pd.merge(master_dataset_new,transaction_type_map,on='transaction_type',how='inner')
master_dataset_new=pd.merge(master_dataset_new1,agent_map,on='agent',how='inner')
########## removing unwanted columns #############
master_dataset_new=master_dataset_new.drop(['agent','transaction_type'], axis=1)

In [12]:
master_dataset_new.head()

,user_id,transaction_date,mcc_group,amount_n26_currency,direction,transaction_type_Id,agent_Id
0,a78884f5e76951188c1e719d4956773a,2016-02-01,0,350,Out,0,0
1,b0333294fef6ff1299102a70ad46b126,2016-02-01,0,202,Out,0,0
2,7b8d2a2780adae0cd0c248e92c1b28dc,2016-02-01,0,291,Out,0,0
3,cc4abaa500f7db4390ae3f02bd36d805,2016-02-01,0,214,Out,0,0
4,49b99d1d5ba028566639e8b3eb7c055b,2016-02-01,0,272,Out,0,0


In [13]:

master_dataset_new['transaction_date'] = pd.to_datetime(master_dataset_new['transaction_date']) 
############# Taking Income(x_train_in) and expense(x_train_out) data seperately #########
x_train_in = master_dataset_new.query('direction == "In"')
print "XTRAIN Income\n\n",x_train_in.head(2)
x_train_out = master_dataset_new.query('direction == "Out"')
print "XTRAIN Expense\n\n",x_train_out.head(2)

XTRAIN Income

                                user_id transaction_date  mcc_group  \
40528  77a52d4da43d0106d321cdc1364bfe01       2016-02-01          0   
40529  f70f8ff785fac332a46323ab174082e2       2016-02-01          0   

       amount_n26_currency direction  transaction_type_Id  agent_Id  
40528                   24        In                    1         0  
40529                   38        In                    1         0  
XTRAIN Expense

                            user_id transaction_date  mcc_group  \
0  a78884f5e76951188c1e719d4956773a       2016-02-01          0   
1  b0333294fef6ff1299102a70ad46b126       2016-02-01          0   

   amount_n26_currency direction  transaction_type_Id  agent_Id  
0                  350       Out                    0         0  
1                  202       Out                    0         0  


In [14]:
print x_train_out.describe()
x_train_in.info()

           mcc_group  amount_n26_currency  transaction_type_Id       agent_Id
count  336421.000000        336421.000000        336421.000000  336421.000000
mean        3.716566            34.282845             4.277718       0.774827
std         4.499277            70.930619             1.635168       0.425956
min         0.000000             3.000000             0.000000       0.000000
25%         1.000000             7.000000             5.000000       1.000000
50%         2.000000            14.000000             5.000000       1.000000
75%         6.000000            30.000000             5.000000       1.000000
max        17.000000          1166.000000             8.000000       2.000000
<class 'pandas.core.frame.DataFrame'>
Int64Index: 72125 entries, 40528 to 407373
Data columns (total 7 columns):
user_id                72125 non-null object
transaction_date       72125 non-null datetime64[ns]
mcc_group              72125 non-null int64
amount_n26_currency    72125 non-null int64

In [15]:
#feature creation:

#I will train this model on last four month data    
######Intialization for the feature creation ########

date=min(master_dataset_new['transaction_date'])
#date=datetime.date(*(int(s) for s in date1.split('-'))) #datetime.date(*(int(s) for s in t.split('-')))
cut_off_date  = date +  datetime.timedelta(days=(60))
print type(cut_off_date),type(date)
##### cut_off_date is being taken for removing the biased data #########
f_train_x_out = []
f_train_y_out = []

f_train_x_in = []
f_train_y_in = []  

sum1 = 0


<class 'pandas._libs.tslib.Timestamp'> <class 'pandas._libs.tslib.Timestamp'>


In [16]:
##### creating Data frame of all users ##########
userAll=pd.DataFrame(master_dataset_new.user_id.unique())


In [ ]:
sum1 = 0
for uid in userAll[0]:
    sum1 +=1
    if(sum1%100==0):
        print 'user processing'
        print sum1
    my_symbol = [uid]
    out_data = x_train_out.query('user_id == @my_symbol ' )
    out_data = out_data.groupby('transaction_date').sum().reset_index()
    in_data  = x_train_in.query('user_id == @my_symbol ' )
    in_data = in_data.groupby('transaction_date').sum().reset_index()  
    for date in in_data['transaction_date']:
      if(date>cut_off_date):
          train_x_out = []
          train_x_in = []
          train_y_out = []      
          train_y_in = []  
                
          date1  = date +  datetime.timedelta(days=(-90))  
          df  = out_data[(out_data['transaction_date']> date1) & (out_data['transaction_date']<= date)]
          f1_out = sum(df['amount_n26_currency'])
          df = in_data[(in_data['transaction_date']> date1) & (in_data['transaction_date']<= date)]
          f1_in = sum(df['amount_n26_currency'])
        
          train_x_out.append(f1_out)
          train_x_out.append(f1_in)
          train_x_in.append(f1_in)
          train_x_in.append(f1_out)
   
        
          mcc_group_in=in_data.query('transaction_date==@date')['mcc_group']
          transaction_type_in=in_data.query('transaction_date==@date')['transaction_type_Id']
          agent_in=in_data.query('transaction_date==@date')['agent_Id']#agent_Id  
        
          train_x_in.append(int(mcc_group_in))
          train_x_in.append(int(transaction_type_in))
          train_x_in.append(int(transaction_type_in))
            ##################
          date1  = date +  datetime.timedelta(days=(-30))  
          df = out_data[(out_data['transaction_date']> date1) & (out_data['transaction_date']<= date)]
          f2_out = sum(df['amount_n26_currency'])
          df = in_data[(in_data['transaction_date']> date1) & (in_data['transaction_date']<= date)]
          f2_in = sum(df['amount_n26_currency'])
          train_x_out.append(f2_out)
          train_x_out.append(f2_in)
          train_x_in.append(f2_in)
          train_x_in.append(f2_out)
        
          date1  = date +  datetime.timedelta(days=(-7))  
          df = out_data[(out_data['transaction_date']> date1) & (out_data['transaction_date']<= date)]
          f3_out = sum(df['amount_n26_currency'])
          df = in_data[(in_data['transaction_date']> date1) & (in_data['transaction_date']<= date)]
          f3_in = sum(df['amount_n26_currency'])
          train_x_out.append(f3_out)
          train_x_out.append(f3_in)
          train_x_in.append(f3_in)
          train_x_in.append(f3_out)
            
          date1  = date +  datetime.timedelta(days=(-2))  
          df = out_data[(out_data['transaction_date']> date1) & (out_data['transaction_date']<= date)]
          f4_out = sum(df['amount_n26_currency'])
          df = in_data[(in_data['transaction_date']> date1) & (in_data['transaction_date']<= date)]
          f4_in = sum(df['amount_n26_currency'])
          train_x_out.append(f4_out)
          train_x_out.append(f4_in)
          train_x_in.append(f4_in)
          train_x_in.append(f4_out)
          
          #date1  = date +  datetime.timedelta(days=(-30))  
          last_30_days = f2_out
          avg_last_90_days = f1_out/3.0
          standard = sum(out_data['amount_n26_currency'])/6.0
          f5_out = (last_30_days-avg_last_90_days)/6.0
          
          last_30_days = f2_in
          avg_last_90_days = f1_in/3.0
          standard = sum(in_data['amount_n26_currency'])/6.0
          f5_in = (last_30_days-avg_last_90_days)/6.0
          train_x_out.append(f5_out)
          train_x_out.append(f5_in)
          train_x_in.append(f5_in)
          train_x_in.append(f5_out)
            
          df = out_data[(out_data['transaction_date'] == date)]
          train_y_out = sum(df['amount_n26_currency'])
          df = in_data[(in_data['transaction_date'] == date)]
          train_y_in = sum(df['amount_n26_currency'])
          f_train_x_in.append(train_x_in)
          f_train_y_in.append(train_y_in)
        
    for date in out_data['transaction_date']:
      if(date>cut_off_date):
          train_x_out = []
          train_x_in = []
          train_y_out = []      
          train_y_in = []  
                
          date1  = date +  datetime.timedelta(days=(-90))  
          df  = out_data[(out_data['transaction_date']> date1) & (out_data['transaction_date']<= date)]
          f1_out = sum(df['amount_n26_currency'])
          df = in_data[(in_data['transaction_date']> date1) & (in_data['transaction_date']<= date)]
          f1_in = sum(df['amount_n26_currency'])
        
          train_x_out.append(f1_out)
          train_x_out.append(f1_in)
          train_x_in.append(f1_in)
          train_x_in.append(f1_out)
          ##########
          mcc_group_out=out_data.query('transaction_date==@date')['mcc_group']
          transaction_type_out=out_data.query('transaction_date==@date')['transaction_type_Id']
          agent_out=out_data.query('transaction_date==@date')['agent_Id']#agent_Id  
        
          train_x_out.append(int(mcc_group_out))
          train_x_out.append(int(transaction_type_out))
          train_x_out.append(int(agent_out))

            ##################
          date1  = date +  datetime.timedelta(days=(-30))  
          df = out_data[(out_data['transaction_date']> date1) & (out_data['transaction_date']<= date)]
          f2_out = sum(df['amount_n26_currency'])
          df = in_data[(in_data['transaction_date']> date1) & (in_data['transaction_date']<= date)]
          f2_in = sum(df['amount_n26_currency'])
          train_x_out.append(f2_out)
          train_x_out.append(f2_in)
          train_x_in.append(f2_in)
          train_x_in.append(f2_out)
        
          date1  = date +  datetime.timedelta(days=(-7))  
          df = out_data[(out_data['transaction_date']> date1) & (out_data['transaction_date']<= date)]
          f3_out = sum(df['amount_n26_currency'])
          df = in_data[(in_data['transaction_date']> date1) & (in_data['transaction_date']<= date)]
          f3_in = sum(df['amount_n26_currency'])
          train_x_out.append(f3_out)
          train_x_out.append(f3_in)
          train_x_in.append(f3_in)
          train_x_in.append(f3_out)
            
          date1  = date +  datetime.timedelta(days=(-2))  
          df = out_data[(out_data['transaction_date']> date1) & (out_data['transaction_date']<= date)]
          f4_out = sum(df['amount_n26_currency'])
          df = in_data[(in_data['transaction_date']> date1) & (in_data['transaction_date']<= date)]
          f4_in = sum(df['amount_n26_currency'])
          train_x_out.append(f4_out)
          train_x_out.append(f4_in)
          train_x_in.append(f4_in)
          train_x_in.append(f4_out)
          
          #date1  = date +  datetime.timedelta(days=(-30))  
          last_30_days = f2_out
          avg_last_90_days = f1_out/3.0
          standard = sum(out_data['amount_n26_currency'])/6.0
          f5_out = (last_30_days-avg_last_90_days)/6.0
          
          last_30_days = f2_in
          avg_last_90_days = f1_in/3.0
          standard = sum(in_data['amount_n26_currency'])/6.0
          f5_in = (last_30_days-avg_last_90_days)/6.0
          train_x_out.append(f5_out)
          train_x_out.append(f5_in)
          train_x_in.append(f5_in)
          train_x_in.append(f5_out)
            
          df = out_data[(out_data['transaction_date'] == date)]
          train_y_out = sum(df['amount_n26_currency'])
          df = in_data[(in_data['transaction_date'] == date)]
          train_y_in = sum(df['amount_n26_currency'])
          #print train_x_out
          f_train_x_out.append(train_x_out)
          f_train_y_out.append(train_y_out)


user processing
100
user processing
200
user processing
300
user processing
400
user processing
500
user processing
600
user processing
700
user processing
800
user processing
900
user processing
1000
user processing
1100
user processing
1200
user processing
1300
user processing
1400
user processing
1500
user processing
1600
user processing
1700
user processing
1800
user processing
1900
user processing
2000
user processing
2100
user processing
2200
user processing
2300
user processing
2400
user processing
2500
user processing
2600
user processing
2700
user processing
2800
user processing
2900
user processing
3000
user processing
3100
user processing
3200
user processing
3300
user processing
3400
user processing
3500
user processing
3600
user processing
3700
user processing
3800
user processing
3900
user processing
4000
user processing
4100
user processing
4200
user processing
4300
user processing
4400
user processing
4500
user processing
4600
user processing
4700
user processing
4800
u

In [20]:
from sklearn import linear_model
lm = linear_model.LinearRegression() 
#regr= LinearRegression()
lm.fit(f_train_x_in, f_train_y_in)

print "linear r squared",lm.score(f_train_x_in,f_train_y_in)
predictions = lm.predict(f_train_x_in)
from sklearn.metrics import mean_squared_error
mean_squared_error(f_train_y_in, predictions)

linear r squared 0.917846939354


3858.9241347532288

In [19]:
pd.DataFrame(f_train_x_in).shape

(42311, 13)

In [21]:
############## Creating the training and testing set ###################
from sklearn.model_selection import train_test_split
X_train_in, X_test_in, y_train_in, y_test_in = train_test_split(f_train_x_in, f_train_y_in, test_size=0.2)

In [22]:
################### trying lm ################
from sklearn import linear_model
lm = linear_model.LinearRegression() 
#regr= LinearRegression()
lm.fit(X_train_in, y_train_in)

prediction_lm=lm.predict(X_test_in)
print "------lm----",lm.score(X_test_in,y_test_in)
print "---lm---",mean_squared_error(y_test_in, prediction_lm)

#regr.score(X_test_in,y_test_in)

################### trying Random Forest ################
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(max_depth=25, random_state=0,n_estimators =15,)
regr.fit(X_train_in, y_train_in)
prediction_randF=regr.predict(X_test_in)
print "---RandF---",mean_squared_error(y_test_in, prediction_randF)
print "---RandF---", regr.score(X_test_in,y_test_in)

################### XgBoost ####################
from xgboost import XGBRegressor 
xgbs= XGBRegressor(
            seed=10,
            n_estimators=500, max_depth=50,
            learning_rate=0.05, subsample=0.8, colsample_bytree=0.75
        )
xgbs.fit(X_train_in, y_train_in)
prediction_xgb=xgbs.predict(X_test_in)
print "---XgBoost---", mean_squared_error(y_test_in, prediction_xgb)  
print "---XgBoost---", xgbs.score(X_test_in,y_test_in)

############## Neural Net Regression ##########
from sklearn.neural_network import MLPRegressor

clfN2 = MLPRegressor(solver='adam', alpha=.001,activation ="relu",
                    hidden_layer_sizes=(128,64,32,16), random_state=2,max_iter=5000)

clfN2.fit(X_train_in, y_train_in)
prediction_neuralNet2 = clfN2.predict(X_test_in)
print "---Neural---",mean_squared_error(y_test_in, prediction_neuralNet2)
print "---Neural---", clfN2.score(X_test_in,y_test_in)

########## Gradient Boosting machine ###########
from sklearn.ensemble import RandomForestRegressor, VotingClassifier,GradientBoostingRegressor,ExtraTreesRegressor
gbr=GradientBoostingRegressor(
            random_state=0, 
            n_estimators=500, max_depth=10,
            learning_rate=0.05, subsample=0.8
        )
gbr.fit(X_train_in, y_train_in)
prediction_gbr=gbr.predict(X_test_in)
#mean_squared_error(y_test_in, prediction_gbr) 
print "---GBM---",mean_squared_error(y_test_in, prediction_gbr) 
print "---GBM---", gbr.score(X_test_in,y_test_in)

------lm---- 0.915104488624
---lm--- 3865.92418701
---RandF--- 3479.04777743
---RandF--- 0.923600276188


/usr/local/ActivePython-2.7.13/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


---XgBoost--- 3636.04635022
---XgBoost--- 0.920152594993
---Neural--- 3892.83594879
---Neural--- 0.914513507615
---GBM--- 3604.5705536
---GBM--- 0.92084380199


In [24]:
############## Creating the training and testing set ###################
from sklearn.model_selection import train_test_split
X_train_out, X_test_out, y_train_out, y_test_out = train_test_split(f_train_x_out, f_train_y_out, test_size=0.2)

In [ ]:
################### trying lm ################
from sklearn import linear_model
lm = linear_model.LinearRegression() 
#regr= LinearRegression()
lm.fit(X_train_out, y_train_out)

prediction_lm=lm.predict(X_test_out)
print "---lm---",lm.score(X_test_out,y_test_out)
print "---lm---",mean_squared_error(y_test_out, prediction_lm)

#regr.score(X_test_out,y_test_out)

################### trying Random Forest ################
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(max_depth=25, random_state=0,n_estimators =15,)
regr.fit(X_train_out, y_train_out)
prediction_randF=regr.predict(X_test_out)
print "---RandF---",mean_squared_error(y_test_out, prediction_randF)
print "---RandF---", regr.score(X_test_out,y_test_out)

################### XgBoost ####################
from xgboost import XGBRegressor 
xgbs= XGBRegressor(
            seed=10,
            n_estimators=500, max_depth=50,
            learning_rate=0.05, subsample=0.8, colsample_bytree=0.75
        )
xgbs.fit(X_train_out, y_train_out)
prediction_xgb=xgbs.predict(X_test_out)
print "---XgBoost---", mean_squared_error(y_test_out, prediction_xgb)  
print "---XgBoost---", xgbs.score(X_test_out,y_test_out)

############## Neural Net Regression ##########
from sklearn.neural_network import MLPRegressor

clfN2 = MLPRegressor(solver='adam', alpha=.001,activation ="relu",
                    hidden_layer_sizes=(128,64,32,16), random_state=2,max_iter=5000)

clfN2.fit(X_train_out, y_train_out)
prediction_neuralNet2 = clfN2.predict(X_test_out)
print "---Neural---",mean_squared_error(y_test_out, prediction_neuralNet2)
print "---Neural---", clfN2.score(X_test_out,y_test_out)

########## Gradient Boosting machine ###########
from sklearn.ensemble import RandomForestRegressor, VotingClassifier,GradientBoostingRegressor,ExtraTreesRegressor
gbr=GradientBoostingRegressor(
            random_state=0, 
            n_estimators=500, max_depth=10,
            learning_rate=0.05, subsample=0.8
        )
gbr.fit(X_train_out, y_train_out)
prediction_gbr=gbr.predict(X_test_out)
#mean_squared_error(y_test_out, prediction_gbr) 
print "---GBM---",mean_squared_error(y_test_out, prediction_gbr) 
print "---GBM---", gbr.score(X_test_out,y_test_out)

---lm--- 0.705834239232
---lm--- 3192.53155236
---RandF--- 3245.60622534
---RandF--- 0.700943840721
---XgBoost--- 3279.52629709
---XgBoost--- 0.697818382585
---Neural--- 3240.3091072
---Neural--- 0.701431926981
